In [0]:

from pyspark.sql.functions import col, current_timestamp

# 1. Lire les tables Silver
prd_df = spark.table("lakehouse.silver.crm_prdd")  # ou la table Silver correspondant à "prd_key_px_cat"
category_df = spark.table("lakehouse.silver.erp_categoryy")  # attention au double "y"

# 2. Left join
gold_df = prd_df.join(
    category_df,
    prd_df["prd_key_px_cat"] == category_df["ID"],
    how="left"
)

# 3. Sélection et transformations
gold_df = gold_df.select(
    col("prd_id").alias("product_nk"),
    col("prd_key"),
    col("prd_name"),
    col("prd_cost").cast("decimal(18,2)").alias("prd_cost"),
    col("prd_start_dt"),
    col("prd_end_dt"),
    col("prd_key_sales"),
    col("prd_line"),
    col("ID").alias("category_id"),
    col("Category").alias("category"),
    col("Subcategory").alias("subcategory"),
    col("Maintenance").alias("maintenance"),
    current_timestamp().alias("load_ts")
)

# 4. Écriture Gold layer
gold_df.write.mode("overwrite").format("delta").saveAsTable("lakehouse.gold.dim_productss")

print("Gold dimension dim_productss created successfully")
